In [40]:
import os
import cv2
import torch
import numpy as np
import torchvision.transforms as transforms
from skimage.morphology import binary_dilation,binary_erosion,convex_hull_object

# if not hasattr(PIL.Image, 'Resampling'):  # Pillow<9.0
#     PIL.Image.Resampling = PIL.Image

import onnxruntime as rt



device = "cpu"


import PIL.Image
def transform_image(image):
    my_transforms = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image_aux = image
    return my_transforms(image_aux).unsqueeze(0).to(device)


st_element = np.ones((3, 3), dtype=bool)


class model:
    def __init__(self):
        self.checkpoint_LC1 = "deeplab-lacker-semi.pth"
        self.checkpoint_LC2 = "unetplusplus-lacker-semi.pth"
        self.checkpoint_LC3 = "deeplab-efficientnet-lacquer.pth"
        self.eye_model='eye_model.onnx'
        self.checkpoint_CNV1 = "deeplab-choroidal-crop.pth"
        self.checkpoint_CNV2 = "deeplab-choroidal-semi.pth"
        self.checkpoint_CNV3 = "deeplab-choroidal-efficientnet-crop.pth"
        self.checkpoint_FS1 = "deeplab-fuchs-semi.pth"
        self.checkpoint_FS2 = "unetplusplus-fuchs-semi.pth"
        self.checkpoint_FS3 = "deeplab-efficientnet-fuchs.pth"
        # The model is evaluated using CPU, please do not change to GPU to avoid error reporting.
        self.device = torch.device("cpu")


    def eye_predict(self,eye_model,image):
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = PIL.Image.fromarray(image)
        image = image.resize((128,128))
        image = np.array(image).astype(np.float32)
        x = np.expand_dims(image, axis=0)
        m = rt.InferenceSession(eye_model,providers=['CPUExecutionProvider'])
        res = m.run(['activation_5'], {"convolution2d_1_input": x})
        if(res[0][0][0]==1):
            return 'd'
        else:
            return 'i'
            
            
    def load(self, dir_path):
        """
        load the model and weights.
        dir_path is a string for internal use only - do not remove it.
        all other paths should only contain the file name, these paths must be
        concatenated with dir_path, for example: os.path.join(dir_path, filename).
        :param dir_path: path to the submission directory (for internal use only).
        :return:
        """
        self.eye_model = os.path.join(dir_path, 'eye_model.onnx')
        
        """checkpoint_path_LC1 = os.path.join(dir_path, self.checkpoint_LC1)
        self.model_LC1 = torch.load(checkpoint_path_LC1, map_location=self.device)
        self.model_LC1.to(self.device)
        self.model_LC1.eval()
        checkpoint_path_LC2 = os.path.join(dir_path, self.checkpoint_LC2)
        self.model_LC2 = torch.load(checkpoint_path_LC2, map_location=self.device)
        self.model_LC2.to(self.device)
        self.model_LC2.eval()
        checkpoint_path_LC3 = os.path.join(dir_path, self.checkpoint_LC3)
        self.model_LC3 = torch.load(checkpoint_path_LC3, map_location=self.device)
        self.model_LC3.to(self.device)
        self.model_LC3.eval()"""
        
        checkpoint_path_CNV1 = os.path.join(dir_path, self.checkpoint_CNV1)
        self.model_CNV1 = torch.load(checkpoint_path_CNV1, map_location=self.device)
        self.model_CNV1.to(self.device)
        self.model_CNV1.eval()
        checkpoint_path_CNV2 = os.path.join(dir_path, self.checkpoint_CNV2)
        self.model_CNV2 = torch.load(checkpoint_path_CNV2, map_location=self.device)
        self.model_CNV2.to(self.device)
        self.model_CNV2.eval()
        checkpoint_path_CNV3 = os.path.join(dir_path, self.checkpoint_CNV3)
        self.model_CNV3 = torch.load(checkpoint_path_CNV3, map_location=self.device)
        self.model_CNV3.to(self.device)
        self.model_CNV3.eval()

        """checkpoint_path_FS1 = os.path.join(dir_path, self.checkpoint_FS1)
        self.model_FS1 = torch.load(checkpoint_path_FS1, map_location=self.device)
        self.model_FS1.to(self.device)
        self.model_FS1.eval()
        
        checkpoint_path_FS2 = os.path.join(dir_path, self.checkpoint_FS2)
        self.model_FS2 = torch.load(checkpoint_path_FS2, map_location=self.device)
        self.model_FS2.to(self.device)
        self.model_FS2.eval()
        
        checkpoint_path_FS3 = os.path.join(dir_path, self.checkpoint_FS3)
        self.model_FS3 = torch.load(checkpoint_path_FS3, map_location=self.device)
        self.model_FS3.to(self.device)
        self.model_FS3.eval()"""
        
    def predict(self, input_image, lesion_type, patient_info_dict):
        """
        perform the prediction given an image and the metadata.
        input_image is a ndarray read using cv2.imread(path_to_image, 1).
        note that the order of the three channels of the input_image read by cv2 is BGR.
        :param input_image: the input image to the model.
        :param lesion_type: a string indicates the lesion type of the input image: 'Lacquer_Cracks' or 'Choroidal_Neovascularization' or 'Fuchs_Spot'.
        :param patient_info_dict: a dictionary with the metadata for the given image,
        such as {'age': 52.0, 'sex': 'male', 'height': nan, 'weight': 71.3},
        where age, height and weight are of type float, while sex is of type str.
        :return: a ndarray indicates the predicted segmentation mask with the shape 800 x 800.
        The pixel value for the lesion area is 255, and the background pixel value is 0.
        """
        
        image = PIL.Image.fromarray(cv2.cvtColor(input_image,cv2.COLOR_BGR2RGB))
        tensor = transform_image(image=image)
        

        with torch.no_grad():
            if lesion_type == 'Lacquer_Cracks':
                pred_mask1 = self.model_LC1(tensor)
                pred_mask2 = self.model_LC2(tensor)
                pred_mask3 = self.model_LC3(tensor)
                pred_mask = torch.argmax((pred_mask1+pred_mask2+pred_mask3)/3,1)
                #pred_mask = torch.argmax(pred_mask1,1)+torch.argmax(pred_mask2,1)
            elif lesion_type == 'Choroidal_Neovascularization':
                mask = np.zeros((800,800), dtype="uint8")
    
                if(self.eye_predict(self.eye_model,input_image)=='i'):
                    cv2.rectangle(mask, (200, 200), (700, 600), 255, -1)
                else:
                    cv2.rectangle(mask, (100, 200), (600, 600), 255, -1)
                masked = cv2.bitwise_and(input_image, input_image, mask=mask)
                pred_mask1 = self.model_CNV1(transform_image(image=PIL.Image.fromarray(cv2.cvtColor(masked,cv2.COLOR_BGR2RGB))))
                pred_mask3 = self.model_CNV3(transform_image(image=PIL.Image.fromarray(cv2.cvtColor(masked,cv2.COLOR_BGR2RGB))))
                pred_mask2 = self.model_CNV2(tensor)
                pred_mask = torch.argmax(pred_mask1,1)+torch.argmax(pred_mask2,1)+torch.argmax(pred_mask3,1)
                #pred_mask = torch.argmax(pred_mask1,1)
            elif lesion_type == 'Fuchs_Spot':
                pred_mask1 = self.model_FS1(tensor)
                pred_mask2 = self.model_FS2(tensor)
                pred_mask3 = self.model_FS3(tensor)
                pred_mask = torch.argmax((pred_mask1+pred_mask2+pred_mask3)/3,1)
                #pred_mask = torch.argmax(pred_mask1,1)+torch.argmax(pred_mask2,1)
        
        mask = np.array(pred_mask.cpu())
        mask[mask==1]=255
        mask[mask==2]=255
        mask[mask==3]=255
        mask[mask==255]=1
        mask=np.reshape(mask,(800,800))
        mask=convex_hull_object(binary_dilation(binary_erosion(mask,st_element),st_element))
        mask=np.reshape(mask,(800*800))
        mask[mask==True]=255
        mask[mask==False]=0
        mask=np.reshape(mask,(800,800))
       
        return mask

In [41]:
m = model()

In [42]:
m.load("models/")

In [43]:
m.predict(cv2.imread('Images/TrainingSet/mmac_task_1_train_0001.png',1),'Choroidal_Neovascularization',None)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [4]:
from imutils import paths
from tqdm import tqdm

resLacker = []
for im in tqdm(paths.list_images('Images/TrainingSet/')):
    pred = m.predict(cv2.imread(im,1),'Lacquer_Cracks',None)
    temp = [im]
    if(len(np.unique(pred))>1):
        cv2.imwrite(im.replace('TrainingSet','Lacker2'),pred)
#         (numLabels, labels, stats, centroids) = cv2.connectedComponentsWithStats(pred.astype('uint8'),4, cv2.CV_8U)
#         temp.append(numLabels-1)
#         k = 0
#         for s in stats[1:]:
#             k=k+s[4]
#         temp.append(k)
#     else:
#         temp.append(0)
#         temp.append(0)
    
#     resLacker.append(temp)

1143it [01:48, 10.50it/s]


In [5]:
from imutils import paths
from tqdm import tqdm

nummax = 0
resChoroidal = []
for im in tqdm(paths.list_images('Images/TrainingSet/')):
    pred = m.predict(cv2.imread(im,1),'Choroidal_Neovascularization',None)
    temp = [im]
    if(len(np.unique(pred))>1):
        cv2.imwrite(im.replace('TrainingSet','Choroidal2'),pred)
    #     (numLabels, labels, stats, centroids) = cv2.connectedComponentsWithStats(pred.astype('uint8'),4, cv2.CV_8U)
    #     temp.append(numLabels-1)  
    #     k = 0
    #     for s in stats[1:]:
    #         k=k+s[4]
    #     temp.append(k)
    # else:
    #     temp.append(0)
    #     temp.append(0)
    # resChoroidal.append(temp)

1143it [01:49, 10.45it/s]


In [12]:
from imutils import paths
from tqdm import tqdm


resFuchs = []
for im in tqdm(paths.list_images('Images/TrainingSet/')):
    pred = m.predict(cv2.imread(im,1),'Fuchs_Spot',None)
    temp = [im]
    if(len(np.unique(pred))>1):
        cv2.imwrite(im.replace('TrainingSet','Fuchs2'),pred)
    #     (numLabels, labels, stats, centroids) = cv2.connectedComponentsWithStats(pred.astype('uint8'),4, cv2.CV_8U)
    #     temp.append(numLabels-1)
    #     k = 0
    #     for s in stats[1:]:
    #         k=k+s[4]
    #     temp.append(k)
    # else:
    #     temp.append(0)
    #     temp.append(0)
    # resFuchs.append(temp)

1143it [01:48, 10.54it/s]


In [15]:
import pandas as pd
dfNew = pd.concat([pd.DataFrame(resFuchs).loc[:, 0:3],pd.DataFrame(resLacker).loc[:,1:3],pd.DataFrame(resChoroidal).loc[:,1:3]],axis=1)

In [16]:
dfNew

,0,1,2,1,2,1,2
0,Images/TrainingSet/mmac_task_1_train_0400.png,0,0,0,0,0,0
1,Images/TrainingSet/mmac_task_1_train_0983.png,0,0,4,180,0,0
2,Images/TrainingSet/mmac_task_1_train_0075.png,0,0,0,0,0,0
3,Images/TrainingSet/mmac_task_1_train_0805.png,0,0,0,0,1,484
4,Images/TrainingSet/mmac_task_1_train_0414.png,0,0,0,0,0,0
...,...,...,...,...,...,...,...
1138,Images/TrainingSet/mmac_task_1_train_0246.png,0,0,0,0,0,0
1139,Images/TrainingSet/mmac_task_1_train_0404.png,0,0,0,0,0,0
1140,Images/TrainingSet/mmac_task_1_train_0847.png,0,0,0,0,0,0
1141,Images/TrainingSet/mmac_task_1_train_0462.png,0,0,1,47,1,235


In [17]:
from sklearn.model_selection import train_test_split

In [18]:
df1 = pd.read_csv('Groundtruths/MMAC2023_Myopic_Maculopathy_Classification_Training_Labels.csv')

In [36]:
df1['numFuchs'] = df1['image'].apply(lambda x : dfNew[dfNew[0]=='Images/TrainingSet/'+x].iloc[:,1].values[0])

In [38]:
df1['sizeFuchs'] = df1['image'].apply(lambda x : dfNew[dfNew[0]=='Images/TrainingSet/'+x].iloc[:,2].values[0])

In [39]:
df1['numLacker'] = df1['image'].apply(lambda x : dfNew[dfNew[0]=='Images/TrainingSet/'+x].iloc[:,3].values[0])
df1['sizeLacker'] = df1['image'].apply(lambda x : dfNew[dfNew[0]=='Images/TrainingSet/'+x].iloc[:,4].values[0])

In [40]:
df1['numChoroidal'] = df1['image'].apply(lambda x : dfNew[dfNew[0]=='Images/TrainingSet/'+x].iloc[:,5].values[0])
df1['sizeChoroidal'] = df1['image'].apply(lambda x : dfNew[dfNew[0]=='Images/TrainingSet/'+x].iloc[:,6].values[0])

In [41]:
df1.to_csv('Groundtruths/MMAC2023_Myopic_Maculopathy_Classification_Training_Labels_new.csv',index=None)

In [20]:
dfNew['myopic_maculopathy_grade'] = dfNew[0].apply(lambda x: df1[df1.image==x[x.rfind('/')+1:]].values[0][1])

In [21]:
train,test=train_test_split(dfNew,test_size=0.2)

In [22]:
train

,0,1,2,1,2,1,2,myopic_maculopathy_grade
901,Images/TrainingSet/mmac_task_1_train_0778.png,0,0,0,0,1,108,0
823,Images/TrainingSet/mmac_task_1_train_0467.png,0,0,0,0,0,0,1
713,Images/TrainingSet/mmac_task_1_train_0151.png,0,0,0,0,0,0,1
114,Images/TrainingSet/mmac_task_1_train_0941.png,0,0,0,0,0,0,2
600,Images/TrainingSet/mmac_task_1_train_1141.png,4,2884,0,0,0,0,4
...,...,...,...,...,...,...,...,...
1005,Images/TrainingSet/mmac_task_1_train_0967.png,2,87,1,82,0,0,2
324,Images/TrainingSet/mmac_task_1_train_0038.png,3,9225,6,195,2,2330,0
60,Images/TrainingSet/mmac_task_1_train_0441.png,2,370,0,0,1,640,2
1014,Images/TrainingSet/mmac_task_1_train_0929.png,1,83,4,323,0,0,2


In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [24]:
x_train,y_train = train.values[:,1:-1],train.values[:,-1]
x_test,y_test = test.values[:,1:-1],test.values[:,-1]

In [25]:
from sklearn.metrics import accuracy_score,cohen_kappa_score

In [26]:
ESTIMATORS = {
    "RandomForestRegressor": RandomForestClassifier(n_estimators=100, max_depth=30, random_state=2),
    "ExtraTreesRegressor": ExtraTreesClassifier(n_estimators=100, max_depth=30, random_state=2),
    "GBR" :GradientBoostingClassifier(random_state=2),
    
}
for name, estimator in ESTIMATORS.items():     
    estimator.fit(x_train, y_train.astype('str'))  
    print("%s\t%f\t%f" % (name,accuracy_score(y_test.astype('str'),estimator.predict(x_test)),
                               cohen_kappa_score(y_test.astype('str'),estimator.predict(x_test),weights='quadratic')))

RandomForestRegressor	0.484716	0.572710
ExtraTreesRegressor	0.493450	0.557014
GBR	0.489083	0.610801


In [103]:
resChoroidal

[['Images/TrainingSet/mmac_task_1_train_0400.png', 0, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0983.png', 0, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0075.png', 0, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0805.png', 484, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0414.png', 0, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0969.png', 0, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0300.png', 0, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_1142.png', 13, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0780.png', 496, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0744.png', 0, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0549.png', 0, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0832.png', 0, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0237.png', 0, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_1143.png', 0, 0, 0, 0, 0],
 ['Images/TrainingSet/mmac_task_1_train_0747.png', 0, 0, 